# SPH6004 Hands On 1
## Gradient Descent, Linear Regression and Logistic Regression

Contents:
  * Recap of linear and logistic regressions
  * Loss functions
  * Solving regression models by using gradient descent methods
  * Examples


In [ ]:
!pip install plotly==5.5.0

In [189]:
# We will be using the following packages

# PyTorch package and submodules
import torch
import torch.nn as nn
from torch.optim import SGD #gradient descent optimizer

# NumPy for math operations, and Pandas for processing tabular data.
import numpy as np
import pandas as pd

# Plotly plotting package
import plotly.graph_objects as go
import plotly.express as px

# We use toy datasets in scikit-learn package
from sklearn.datasets import load_diabetes, load_breast_cancer

## Recap of linear regression

*Linear Regression* is one of the simplest machine learning model for predicting **continuous outcomes** (e.g. time, length). As shown in the figure below, LR tries to learn the statistics (patterns) that exist in a dataset by trying to draw the best fitted **line** to the data.

<p align="center">
    <img width="75%" src="https://upload.wikimedia.org/wikipedia/commons/3/3a/Linear_regression.svg"> 
</p>

### Model formulation

Linear regression is a supervised learning model. We are given a sequence of **independent variables** $(\mathbf x^{(i)})_{i=1}^N$ and the corresponding **dependent variables/labels** $(y^{(i)})_{i=1}^N$.

* Each dependent variable $\mathbf x^{(i)}$ is a $M$-dimensional vector of the form

$$
\mathbf x^{(i)} = (x_1^{(i)}, x_2^{(i)}, \cdots, x_M^{(i)})
$$

* Each label $y^{(i)}$ is a real number.
* Linear regression model has the form (for convenience we define $x^{(i)}_0=1$)

\begin{equation}
h_{\theta}(\mathbf{x}^{(i)}) = \theta_{0} + \theta_{1}x_{1} + ... + \theta_{M}x_{M} = \sum_{k = 0}^{M}{ \theta_{k} x_{k}^{(i)} },
\end{equation}

where the $\theta_i$'s are parameters of the model. The term $\theta_{0}$ is called the **intercept** or **bias** term of the model.

Our goal is to find the 'best' parameters $\hat{\theta}_i$ such that $h_{\hat{\theta}}(\mathbf x^{(i)})\sim y^{(i)}$.

### Loss function

We find $\hat{\theta}$ by *minimizing* the **Mean Square Error** $J(\theta)$ between $h_{\theta}(\mathbf{x})$ and $y$:

\begin{equation}
J_{\text{MSE}}(\theta) =  \frac{1}{2N} \sum_{i = 1}^{N} \left( h_\theta(\mathbf{x}^{(i)}) - y^{(i)} \right)^2.
\end{equation}

This function $J(\theta)$ is called a **loss function** for our linear regression model.


## Recap of Logistic Regression

Logistic regression is a binary **classification** model that fits data $\{(\mathbf x^{(i)}, y^{(i)})\}$ with binary labels $y^{(i)}\in \{0,1\}$. Mathematically the model with parameter $\theta$ has the form

$$
f_\theta(\mathbf x) = \sigma(h_\theta(\mathbf x)) = \sigma(\theta_{0} + \theta_{1}x_{1} + ... + \theta_{M}x_{M}) = \sigma\left (\sum_{k = 0}^{M}{ \theta_{k} x_{k} }\right ),
$$

where $\sigma$ is the **sigmoid function** defined by

$$
\sigma(\xi) = \frac{1}{1+\exp(-\xi)}.
$$

<p align="center">
    <img width="50%" src="https://miro.medium.com/max/1280/0*gKOV65tvGfY8SMem.png"> 
</p>

* The sigmoid function $\sigma$ has range in $(0,1)$.
* Usually $\sigma(h_\theta(\mathbf x))$ is interpreted as the probability of $\mathbf x$ being in class with label $y=1$.
    * To make binary predictions, one can set a threshold $T=0.5$: the prediction is $0$ if $\sigma(h_\theta(\mathbf x))\leq 0.5$, and the prediction is $1$ if $\sigma(h_\theta(\mathbf x))>0.5$.

### Loss function

For the logistic regression model, one usually uses the **binary cross entropy** as the loss function

$$
J_{\text{BCE}}(\theta) = \sum_{i=1}^{N}\left(-  y^{(i)}\log(f_\theta(\mathbf x)) - (1-y^{(i)})\log (1 - f_\theta(\mathbf x)) \right).
$$

* One *minimizes* $J_{\text{BCE}}$ to find the best parameter $\theta$.
* For data with class label $y=1$, minimizing $J_{\text{BCE}}$ is equivalent to maximizing $f_{\theta}$ (recall this is the probability that $\mathbf x$ belong to class $y=1$).
* For data with class label $y=0$, minimizing $J_{\text{BCE}}$ is equivalent to maximizing $(1-f_{\theta})$ (recall this is the probability that $\mathbf x$ belong to class $y=0$).


## Gradient descent method for solving optimization problems

Given a model with parameter $\theta$ and loss function $J$, one usually estimates the best parameter $\hat{\theta}$ by minimizing $J$:

$$
\hat{\theta} = \argmin_{\theta} J(\theta).
$$

The image below illustrates the plot of a loss function $J$ with respect to a 2-dimensional parameter $\theta = (\theta_0,\theta_1)$.
* Gradient of $J$, denoted by $\nabla_\theta J$, gives direction of $\theta$ that leads to fastest *increase* of $J$.
* Gradient descent method minimizes $J$ by following *opposite* direction of gradient of $J$.
* We only follow the downward direction a short time, controlled by *learning* rate $\eta$, and then re-check the gradient direction at the new location.


<img src="https://hackernoon.com/hn-images/1*f9a162GhpMbiTVTAua_lLQ.png" alt="Drawing" stylewidth="200" height="200"/>

### Steps for gradient descent

We start from a randomly drawn parameter $\theta_0$. For $t=0,1,2,\cdots$ we repeat the following steps
1.  Evaluate gradient of the loss function at time $t$, i.e. $\nabla_\theta J(\theta_t)$.
2.  We update the parameter by
    $$ \theta_{t+1} \leftarrow \theta_t-\eta*\nabla_\theta J(\theta_t).$$

## Examples

For this tutorial, we will use Python with the package [PyTorch](https://pytorch.org/) to build linear and logistic regression models. PyTorch is a Deep learning framework developed by facebook to accelerate Deep learning research.

We will first use encapsulated APIs to build models, and then scratch a little deeper to see how things work behind the APIs.

### Example -- linear regression

We use the diabetes dataset from `scikit-learn` package to build our linear regression model. Our aim is to predict a quantitative measure of disease progression one year after baseline. There are 10 independent variables which have already been normalized (to have zero mean):
* age age in years
* sex
* bmi body mass index
* bp average blood pressure
* s1 tc, total serum cholesterol
* s2 ldl, low-density lipoproteins
* s3 hdl, high-density lipoproteins
* s4 tch, total cholesterol / HDL
* s5 ltg, possibly log of serum triglycerides level
* s6 glu, blood sugar level

In [16]:
X_df, y_df = load_diabetes(return_X_y=True, as_frame=True)

# In total we have 442 observations
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   age     442 non-null    float64
 1   sex     442 non-null    float64
 2   bmi     442 non-null    float64
 3   bp      442 non-null    float64
 4   s1      442 non-null    float64
 5   s2      442 non-null    float64
 6   s3      442 non-null    float64
 7   s4      442 non-null    float64
 8   s5      442 non-null    float64
 9   s6      442 non-null    float64
dtypes: float64(10)
memory usage: 34.7 KB


In [179]:
X_df.head(3)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930


In [187]:
# We convert dataframe to PyTorch tensor datatype,
# and then split it into training and testing parts.
X = torch.tensor(X_df.to_numpy(),dtype=torch.float32)
m,n = X.shape
y = torch.tensor(y_df.to_numpy(),dtype=torch.float32).reshape(m,1)

# We use an approx 6:4 train test splitting
cases = ['train','test']
case_list = np.random.choice(cases,size=X.shape[0],replace=True,p=[0.6,0.4])
X_train = X[case_list=='train']
X_test = X[case_list=='test']
y_train = y[case_list=='train']
y_test = y[case_list=='test']

In [188]:
print('Training input has size: ',X_train.shape)

Training input has size:  torch.Size([273, 10])


In PyTorch, data are stored as `torch.tensor` datatype. Models in PyTorch expect the following:
* the leading dimension of data is the index of this data, and 
* the rest of the dimensions are dimension of each data point

We use the build-in linear model, mean square error loss, as well as gradient descent method in PyTorch.

In [157]:
# Linear regression is a linear model.
# We have 10 dimensional inputs and 1 dimensional outputs.
# We want to have the bias term in our model.

h = torch.nn.Linear(
    in_features=10,
    out_features=1,
    bias=True
)

# parameters of our model 'h' are 
# randomly initialized, which are stored in
print(h.weight)
print(h.bias)

Parameter containing:
tensor([[-0.2803,  0.0228,  0.3026,  0.2506, -0.2252, -0.1651,  0.1880, -0.2211,
         -0.2607,  0.3008]], requires_grad=True)
Parameter containing:
tensor([-0.1965], requires_grad=True)


In [175]:
h = torch.nn.Linear(
    in_features=10,
    out_features=1,
    bias=True
)

# For torch SGD, we need to tell it which parameter we what to optimize.
GD_optimizer = torch.optim.SGD(lr=0.05, params=h.parameters())
J_MSE = torch.nn.MSELoss()

# Apply gradient descent 100 times
nIter = 10000
printInterval = 500

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = h(X_train)
    loss = J_MSE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1) % printInterval) == 0:
        # We take square root of MSE (PyTorch internally averaged in J_MSE)
        # so that scale of printout is same as scale of y values.
        print('Iter {} : average rooted training MSE {:.3f}'.format(i+1,torch.sqrt(loss).item()))
    
    

Iter 1 : average rooted training MSE 170.551
Iter 500 : average rooted training MSE 67.624
Iter 1000 : average rooted training MSE 62.659
Iter 1500 : average rooted training MSE 59.861
Iter 2000 : average rooted training MSE 58.162
Iter 2500 : average rooted training MSE 57.044
Iter 3000 : average rooted training MSE 56.255
Iter 3500 : average rooted training MSE 55.670
Iter 4000 : average rooted training MSE 55.224
Iter 4500 : average rooted training MSE 54.877
Iter 5000 : average rooted training MSE 54.604
Iter 5500 : average rooted training MSE 54.388
Iter 6000 : average rooted training MSE 54.216
Iter 6500 : average rooted training MSE 54.078
Iter 7000 : average rooted training MSE 53.969
Iter 7500 : average rooted training MSE 53.880
Iter 8000 : average rooted training MSE 53.810
Iter 8500 : average rooted training MSE 53.753
Iter 9000 : average rooted training MSE 53.706
Iter 9500 : average rooted training MSE 53.669
Iter 10000 : average rooted training MSE 53.639


In [176]:
with torch.no_grad():
    test_pred = h(X_test)

fig = go.Figure(
    data=go.Scatter(
        x=y_test.squeeze(),
        y=test_pred.squeeze(),
        mode='markers'
    )
)
fig.update_xaxes(
    title='true value'
  )
fig.update_yaxes(
    title='predicted value',
    scaleanchor='x',
    scaleratio=1
  )

### Example -- logistic regression

We use the Breast Cancer Wisconsin (Diagnostic) Dataset provided in the Python package `sklearn` to demonstrate training a logistic model. Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass to describe charateristics of the cell nulcei present in the image.

- 569 instances
- dependent variable $\mathbf x$ has 30 dimensions
    - radius
    - texture, smoothness
    - perimeter, area
    - ...
- label $y$ has two classes
    - $0$ for benign (357 instances)
    - $1$ for malignant (212 instances)

In [257]:
X_raw, y_df = load_breast_cancer(return_X_y=True, as_frame=True)
X_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [258]:
# Question: What happens if you don't normalize your data? Give it a try.

X_df = (X_raw-X_raw.mean())/X_raw.std()
X_df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,...,5.690000e+02,5.690000e+02,5.690000e+02,569.000000,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02,5.690000e+02
mean,-1.311195e-16,6.243785e-17,-1.248757e-16,-2.185325e-16,-8.366672e-16,1.998011e-16,3.746271e-17,-4.995028e-17,1.748260e-16,4.838933e-16,...,-8.241796e-16,1.248757e-17,-3.496520e-16,0.000000,-2.122887e-16,-3.621395e-16,8.741299e-17,2.122887e-16,2.622390e-16,-5.744282e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.027864e+00,-2.227289e+00,-1.982759e+00,-1.453164e+00,-3.109349e+00,-1.608721e+00,-1.113893e+00,-1.260710e+00,-2.741705e+00,-1.818265e+00,...,-1.725382e+00,-2.222039e+00,-1.691872e+00,-1.221348,-2.680337e+00,-1.442609e+00,-1.304683e+00,-1.743529e+00,-2.159060e+00,-1.600431e+00
25%,-6.887793e-01,-7.253249e-01,-6.913472e-01,-6.666089e-01,-7.103378e-01,-7.464292e-01,-7.430941e-01,-7.372951e-01,-7.026215e-01,-7.220040e-01,...,-6.743279e-01,-7.479711e-01,-6.889721e-01,-0.641571,-6.906227e-01,-6.804845e-01,-7.558491e-01,-7.557349e-01,-6.412994e-01,-6.913035e-01
50%,-2.148925e-01,-1.045442e-01,-2.357726e-01,-2.949274e-01,-3.486040e-02,-2.217454e-01,-3.419391e-01,-3.973715e-01,-7.156354e-02,-1.781226e-01,...,-2.688030e-01,-4.347738e-02,-2.857288e-01,-0.340881,-4.680159e-02,-2.692639e-01,-2.180402e-01,-2.232725e-01,-1.272975e-01,-2.162538e-01
75%,4.689800e-01,5.836621e-01,4.992377e-01,3.631877e-01,6.356397e-01,4.934227e-01,5.255994e-01,6.463664e-01,5.303125e-01,4.705693e-01,...,5.215568e-01,6.577623e-01,5.398040e-01,0.357275,5.970195e-01,5.391944e-01,5.306742e-01,7.118836e-01,4.497425e-01,4.503661e-01
max,3.967796e+00,4.647799e+00,3.972634e+00,5.245913e+00,4.766717e+00,4.564409e+00,4.239858e+00,3.924477e+00,4.480808e+00,4.906602e+00,...,4.090590e+00,3.882489e+00,4.283568e+00,5.924959,3.951897e+00,5.108382e+00,4.696536e+00,2.683516e+00,6.040726e+00,6.840837e+00


As before, we transform data to `torch.tensor` and do a train test split.

In [259]:
# We convert dataframe to PyTorch tensor datatype,
# and then split it into training and testing parts.
X = torch.tensor(X_df.to_numpy(),dtype=torch.float32)
m,n = X.shape
y = torch.tensor(y_df.to_numpy(),dtype=torch.float32).reshape(m,1)

# We use an approx 6:4 train test splitting
cases = ['train','test']
case_list = np.random.choice(cases,size=X.shape[0],replace=True,p=[0.6,0.4])
X_train = X[case_list=='train']
X_test = X[case_list=='test']
y_train = y[case_list=='train']
y_test = y[case_list=='test']

In [268]:
h = torch.nn.Linear(
    in_features=n,
    out_features=1,
    bias=True
)
sigma = torch.nn.Sigmoid()

# Logistic model is linear+sigmoid
f = torch.nn.Sequential(
    h,
    sigma
)

J_BCE = torch.nn.BCELoss()
GD_optimizer = torch.optim.SGD(lr=0.001,params=f.parameters())

nIter = 10000
printInterval = 1000

for i in range(nIter):
    GD_optimizer.zero_grad()
    pred = f(X_train)
    loss = J_BCE(pred,y_train)
    loss.backward()
    GD_optimizer.step()
    if i == 0 or ((i+1)%printInterval) == 0:
        print('Iter {}: average BCE loss is {:.3f}'.format(i+1,loss.item()))

Iter 1: average BCE loss is 0.620
Iter 1000: average BCE loss is 0.260
Iter 2000: average BCE loss is 0.197
Iter 3000: average BCE loss is 0.168
Iter 4000: average BCE loss is 0.151
Iter 5000: average BCE loss is 0.139
Iter 6000: average BCE loss is 0.131
Iter 7000: average BCE loss is 0.124
Iter 8000: average BCE loss is 0.119
Iter 9000: average BCE loss is 0.115
Iter 10000: average BCE loss is 0.111


In [293]:
# Test on test data

threshold = 0.5

with torch.no_grad():
    pred_test = f(X_test)

binary_pred = np.where(pred_test.squeeze()>threshold,'Malignant','Benign')
label = np.where(y_test.squeeze()>0.5,'Malignant','Benign')
acc = (binary_pred==label).sum()/binary_pred.shape[0]
print('Accuracy on test dataset is {:.2f}%'.format(acc*100))

Accuracy on test dataset is 98.61%


In [294]:
pd.crosstab(
    index=label,
    columns=binary_pred,
    rownames=['Label'],
    colnames=['Pred']
)

Pred,Benign,Malignant
Label,,
Benign,79,2
Malignant,1,134


In [297]:
# Use plotly express (px) to visualize test results.
# px expects DataFrame input

pred_df = pd.DataFrame(
    {
        'pred_probability':pred_test.squeeze(),
        'label':label
    }
)
fig = px.scatter(data_frame=pred_df,y='pred_probability',color='label')
fig